# Predicting the survival of Titanic passengers

The goal of this project is to investigate the Titanic passengers dataset and predict, if the individual will survive or not.

# Loading the data

First, let's import some libraries and take a look at our data.


In [75]:
import pandas as pd
import numpy as np

td = pd.read_csv('titanic_data.csv')

td.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


# Preprocessing the data

We can see a few columns that we certainly don't need: 'PassengerId', 'Name', 'Ticket' and 'Cabin'. 
Let's get rid of them.

In [76]:
td = td.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis = 1)
td.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


Now we need to convert our categorical data in 'Sex' and 'Embarked' columns to numerical one.

In [77]:
td['Male'] = (td['Sex']=='male').apply(int)
td['Female'] = (td['Sex']=='female').apply(int)

td['PortS'] = (td['Embarked']=='S').apply(int)
td['PortC'] = (td['Embarked']=='C').apply(int)
td['PortQ'] = (td['Embarked']=='Q').apply(int)

td = td.drop(['Sex', 'Embarked'], axis = 1)

td.head() 

,Survived,Pclass,Age,SibSp,Parch,Fare,Male,Female,PortS,PortC,PortQ
0,0,3,22.0,1,0,7.2500,1,0,1,0,0
1,1,1,38.0,1,0,71.2833,0,1,0,1,0
2,1,3,26.0,0,0,7.9250,0,1,1,0,0
3,1,1,35.0,1,0,53.1000,0,1,1,0,0
4,0,3,35.0,0,0,8.0500,1,0,1,0,0


I noticed that some passengers have NaN age, but first let's have a look at the correlation.

In [78]:
correlation = td.corr(method='pearson')
print(correlation.loc['Survived'])

Survived    1.000000
Pclass     -0.338481
Age        -0.077221
SibSp      -0.035322
Parch       0.081629
Fare        0.257307
Male       -0.543351
Female      0.543351
PortS      -0.155660
PortC       0.168240
PortQ       0.003650
Name: Survived, dtype: float64


As the correlation is not too strong I will simply replace all NaN values with the mean age.

In [79]:
td['Age'].fillna(td['Age'].mean(), inplace=True)

# Training the model

Now everything we have to do is to convert our dataframe to numpy arrays.

In [80]:
x = td.drop(['Survived'], axis = 1).values
y = td['Survived'].values

Train the model.

In [81]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from xgboost import XGBClassifier

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15)
reg = XGBClassifier(booster='gbtree')
reg.fit(x_train, y_train)

print("Accuracy: ", round(reg.score(x_test, y_test), 3)*100,"%")

Accuracy:  82.8 %
